In [1]:
print("Checking integration with Google Colab.")

Checking integration with Google Colab.


In [ ]:
!pip install pretty_midi
import pretty_midi

In [3]:
midi_data = pretty_midi.PrettyMIDI('data/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_06_Track06_wav.midi')
# Print an empirical estimate of its global tempo
print(midi_data.estimate_tempo())

206.4516129032256
